# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 0.5226444731224815),
 (1, 0.5226444731224815),
 (2, 0.5226444731224815),
 (3, 0.5226444731224815),
 (4, 0.5226444731224815)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18)]),
 (1, [('a', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.016535735578211974),
   (None, 0.02455478407184475),
   (None, 0.10325025526471232),
   (None, 0.16166203305462756),
   (None, 0.1979611292423713),
   (None, 0.23510219865151405),
   (None, 0.25630735292293005),
   (None, 0.31189175632597455),
   (None, 0.33187378685153346),
   (None, 0.3599544412281399),
   (None, 0.3642368170351141),
   (None, 0.3786103708622719),
   (None, 0.4300970857184483),
   (None, 0.44258262681147875),
   (None, 0.4609328836815918),
   (None, 0.46567672824225215),
   (None, 0.4824456283605133)]),
 (1,
  [(None, 0.5287026371356531),
   (None, 0.5287937085592166),
   (None, 0.5550629881943973),
   (None, 0.6139059652137536),
   (None, 0.6180793588834096),
   (None, 0.6937529289640818),
   (None, 0.7649601332048515),
   (None, 0.8437879283869095),
   (None, 0.8574015034678819),
   (None, 0.8843089688413326),
   (None, 0.8845253752286801),
   (None, 0.9520465627224665),
   (None, 0.9766608475187599)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
from typing import Iterator
import numpy as np

input_values = np.random.randint(0, 6, 10)
print("input_values:", input_values)

def RECORDREADER():
    for i in range(len(input_values)):
        yield (None, input_values[i])

def MAP(_, value: int):
    yield (None, value)

def REDUCE(_, values: Iterator[int]):
    max_value = 0
    is_first = True
    for value in values:
        if is_first:
            max_value = value
            is_first = False
        
        max_value = max(max_value, value)
    
    yield (None, max_value)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input_values: [4 4 1 5 4 4 0 4 4 2]


[(None, 5)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def RECORDREADER():
    # [1, 2, 3, 4, 5]
    return [(None, i) for i in range(1, 6)]

def MAP(_, value: int):
    yield (None, value)

def REDUCE(_, values: Iterator[int]):
    values_sum = 0
    amount = 0

    for value in values:
        values_sum += value
        amount += 1
    
    if amount != 0:
        yield (None, values_sum / amount)
    else:
        yield(None, 0)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, 3.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
# Число может быть в группе 1, 2 или 3
# Через каждые три числа число будет помещаться в ту же группу
# Например, в группе 1 будут 1, 4, 7, 10 и т.д.
def RECORDREADER():
    return [(i % 3 + 1, i + 1) for i in range(15)]


def group_by_key(iterable):
    sorted_iterable = sorted(iterable, key=lambda x: x[0])

    result = []
    key_index = -1
    current_key = None

    for key, val in sorted_iterable:
        if key != current_key:
            result.append((key, [val]))
            current_key = key
            key_index += 1
        else:
            result[key_index][1].append(val)

    return result

input = RECORDREADER()

print("input: ", input)
print(group_by_key(input))

input:  [(1, 1), (2, 2), (3, 3), (1, 4), (2, 5), (3, 6), (1, 7), (2, 8), (3, 9), (1, 10), (2, 11), (3, 12), (1, 13), (2, 14), (3, 15)]
[(1, [1, 4, 7, 10, 13]), (2, [2, 5, 8, 11, 14]), (3, [3, 6, 9, 12, 15])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(idx_offset, split):
        for (doc_id, document) in enumerate(split):
            # Каждый документ становится строкой без переносов
            yield (doc_id + idx_offset, document.replace('\n', ' '))

    split_size =  int(np.ceil(len(documents)/maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(i, documents[i:i+split_size])

def MAP(doc_id:int, flatted_doc:str):
    for word in flatted_doc.split(' '):
        yield (doc_id, word)

def REDUCE(doc_id:str, words:Iterator[str]):
    yield (doc_id, list(set(words)))

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0,
  [(0, ['', 'what', 'is', 'it']),
   (2, ['', 'banana', 'is', 'it', 'a']),
   (4, ['', 'what', 'is', 'it'])]),
 (1,
  [(1, ['', 'what', 'is', 'it']),
   (3, ['', 'what', 'is', 'it']),
   (5, ['', 'banana', 'is', 'it', 'a'])])]

# Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            yield (None, (x, y))

def C(t: tuple):
    return sum(t) < 5

def MAP(_, value: tuple):
    if C(value):
        yield (value, value)

def REDUCE(key: tuple, value: Iterator[tuple]):
    yield (key, value)

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [(None, (1, 1)), (None, (1, 2)), (None, (1, 3)), (None, (2, 1)), (None, (2, 2)), (None, (2, 3)), (None, (3, 1)), (None, (3, 2)), (None, (3, 3))]


[((1, 1), [(1, 1)]),
 ((1, 2), [(1, 2)]),
 ((1, 3), [(1, 3)]),
 ((2, 1), [(2, 1)]),
 ((2, 2), [(2, 2)]),
 ((3, 1), [(3, 1)])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
S = set([1, 2])

def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            # Создаем по две штуки, чтобы проверить дедублирование в reduce
            for _ in range(2):
                yield (None, (x, y))

def MAP(_, value: tuple):
    result = []
    for val in value:
        if val in S:
            result.append(val)
    
    result = tuple(result)

    yield (result, result)

def REDUCE(key: tuple, value: Iterator[tuple]):
    yield (key, list(set(value)))

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [(None, (1, 1)), (None, (1, 1)), (None, (1, 2)), (None, (1, 2)), (None, (1, 3)), (None, (1, 3)), (None, (2, 1)), (None, (2, 1)), (None, (2, 2)), (None, (2, 2)), (None, (2, 3)), (None, (2, 3)), (None, (3, 1)), (None, (3, 1)), (None, (3, 2)), (None, (3, 2)), (None, (3, 3)), (None, (3, 3))]


[((1, 1), [(1, 1)]),
 ((1, 2), [(1, 2)]),
 ((1,), [(1,)]),
 ((2, 1), [(2, 1)]),
 ((2, 2), [(2, 2)]),
 ((2,), [(2,)]),
 ((), [()])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            # Создаем по две штуки, чтобы проверить дедублирование в reduce
            for _ in range(2):
                yield (None, (x, y))

def MAP(_, value: tuple):
    yield (value, value)

def REDUCE(key: tuple, value: Iterator[tuple]):
    yield (key, list(set(value)))

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [(None, (1, 1)), (None, (1, 1)), (None, (1, 2)), (None, (1, 2)), (None, (1, 3)), (None, (1, 3)), (None, (2, 1)), (None, (2, 1)), (None, (2, 2)), (None, (2, 2)), (None, (2, 3)), (None, (2, 3)), (None, (3, 1)), (None, (3, 1)), (None, (3, 2)), (None, (3, 2)), (None, (3, 3)), (None, (3, 3))]


[((1, 1), [(1, 1)]),
 ((1, 2), [(1, 2)]),
 ((1, 3), [(1, 3)]),
 ((2, 1), [(2, 1)]),
 ((2, 2), [(2, 2)]),
 ((2, 3), [(2, 3)]),
 ((3, 1), [(3, 1)]),
 ((3, 2), [(3, 2)]),
 ((3, 3), [(3, 3)])]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            yield (None, (x, y))
            # Создаём пересечение
            if x == 1 and y == 2:
                yield (None, (x, y))
            if x == 3 and y == 1:
                yield (None, (x, y))
            

def MAP(_, value: tuple):
    yield (value, value)

def REDUCE(key: tuple, value: Iterator[tuple]):
    if len(value) >= 2:
        yield (key, list(set(value)))

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [(None, (1, 1)), (None, (1, 2)), (None, (1, 2)), (None, (1, 3)), (None, (2, 1)), (None, (2, 2)), (None, (2, 3)), (None, (3, 1)), (None, (3, 1)), (None, (3, 2)), (None, (3, 3))]


[((1, 2), [(1, 2)]), ((3, 1), [(3, 1)])]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            yield ((x, y), (x + y) % 2)
            # Создаём пересечение
            if x == 1 and y == 2:
                yield ((x, y), (x + y + 1) % 2)
            if x == 3 and y == 1:
                yield ((x, y), (x + y + 1) % 2)
            

def MAP(t, id):
    yield (t, id)

def REDUCE(t, id):
    if id == [0]:
        yield (t, t)

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [((1, 1), 0), ((1, 2), 1), ((1, 2), 0), ((1, 3), 0), ((2, 1), 1), ((2, 2), 0), ((2, 3), 1), ((3, 1), 0), ((3, 1), 1), ((3, 2), 1), ((3, 3), 0)]


[((1, 1), (1, 1)), ((1, 3), (1, 3)), ((2, 2), (2, 2)), ((3, 3), (3, 3))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            yield ((x, y), (x + y) % 2)
            # Создаём пересечение
            if x == 1 and y == 2:
                yield ((x, y), (x + y + 1) % 2)
            if x == 3 and y == 1:
                yield ((x, y), (x + y + 1) % 2)

def MAP(t, id):
    if id == [0]:
        yield ((id, t[0]), t[1])
    else: 
        yield ((id, t[1]), t[0])

def REDUCE(t:Iterator[int], id:int):
    res = []
    for v in t:
        res += [v]
    yield tuple(res)


print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [((1, 1), 0), ((1, 2), 1), ((1, 2), 0), ((1, 3), 0), ((2, 1), 1), ((2, 2), 0), ((2, 3), 1), ((3, 1), 0), ((3, 1), 1), ((3, 2), 1), ((3, 3), 0)]


[(0, 1), (1, 2), (0, 2), (0, 3), (1, 1), (1, 3)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
def RECORDREADER():
    for x in range(1, 4):
        for y in range(1, 4):
            yield (None, ((x + y) % 2, x, y))
            # Создаём пересечение
            if x == 1 and y == 2:
                yield (None, ((x + y + 1) % 2, x, y))
            if x == 3 and y == 1:
                yield (None, ((x + y + 1) % 2, x, y))

def MAP(_, t):
    yield (t[0], t[1])

def REDUCE(k, v:Iterator[int]):
    yield (k, sum(v))

print("input", list(RECORDREADER()))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

input [(None, (0, 1, 1)), (None, (1, 1, 2)), (None, (0, 1, 2)), (None, (0, 1, 3)), (None, (1, 2, 1)), (None, (0, 2, 2)), (None, (1, 2, 3)), (None, (0, 3, 1)), (None, (1, 3, 1)), (None, (1, 3, 2)), (None, (0, 3, 3))]


[(0, 11), (1, 11)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [32]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [33]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(small_mat.shape[0]):
        yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
    (i, k) = key
    el_value = 0
    for v in values:
        el_value += v
    yield ((i, k), el_value)

Проверьте своё решение

In [34]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [35]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [36]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            yield ((0, i, j), small_mat[i, j])
    
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((1, j, k), big_mat[j, k])

def MAP_JOIN(k1, v1):
    (mat_num, i, j) = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

def MAP_MUL(k1, v1):
    (i, k) = k1
    yield ((i, k), v1)

def REDUCE_MUL(key, values):
    res_el_value = 0
    for v in values:
        res_el_value += v
    yield (key, res_el_value)

def GET_JOINED():
    for j in joined:
        yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [37]:
I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

def INPUTFORMAT():
    first_mat = []
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            first_mat.append(((0, i, j), small_mat[i, j]))
    yield first_mat
    
    second_mat = []
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            second_mat.append(((1, j, k), big_mat[j, k]))
    yield second_mat

def MAP_JOIN(k1, v1):
    (mat_num, i, j) = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
    for j in joined:
        print("aa", j)
        yield j[1]

def MAP_MUL(k1, v1):
    yield (k1, v1)

def REDUCE_MUL(key, values):
    res_val = 0
    for v in values:
        res_val += v
    yield (key, res_val)

maps = 4
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
    for v in p[1]:
        solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.01858818957538684), ((0, 1), 0.09464512732337645), ((0, 2), 0.11276168064276942), ((0, 3), 0.005953812634082777), ((0, 4), 0.09908030737125365), ((0, 5), 0.006741886004689387), ((0, 6), 0.11334151671400203), ((0, 7), 0.03144348237437576), ((0, 8), 0.0885625891446396), ((0, 9), 0.06449699227494987), ((0, 10), 0.007458238287862348), ((0, 11), 0.01007903487455153), ((0, 12), 0.10027251551213788), ((0, 13), 0.03172512751694138), ((0, 14), 0.01933351758998207), ((0, 15), 0.1141274366014224), ((0, 16), 0.0019948698988518612), ((0, 17), 0.09938877833192832), ((0, 18), 0.01128250566019917), ((0, 19), 0.039368206141466994), ((0, 20), 0.08358611389193467), ((0, 21), 0.07280390238185466), ((0, 22), 0.10374028613181384), ((0, 23), 0.008246834255902268), ((0, 24), 0.03934582997300276), ((0, 25), 0.04241739344779944), ((0, 26), 0.07609298357342863), ((0, 27), 0.04606955041296814), ((0, 28), 0.03830417406371149), ((0, 29), 0.0765713016139

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [38]:
I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

def INPUTFORMAT():
    first_mat = []
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            first_mat.append(((0, i, j), small_mat[i, j]))
    
    global maps
    split_size = int(np.ceil(len(first_mat) / maps))
    
    for i in range(0, len(first_mat), split_size):
        yield first_mat[i:i + split_size]
    
    second_mat = []
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            second_mat.append(((1, j, k), big_mat[j, k]))
    
    split_size = int(np.ceil(len(second_mat) / maps))
    
    for i in range(0, len(second_mat), split_size):
        yield second_mat[i:i + split_size]

def MAP_JOIN(k1, v1):
    (mat_num, i, j) = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
    for j in joined:
        print("aa", j)
        yield j[1]

def MAP_MUL(k1, v1):
    yield (k1, v1)

def REDUCE_MUL(key, values):
    res_val = 0
    for v in values:
        res_val += v
    yield (key, res_val)

maps = 3
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
    for v in p[1]:
        solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.5168650765243704), ((0, 1), 0.13763305249337288), ((0, 2), 0.30041329893082086), ((0, 3), 0.40195312233481945), ((0, 4), 0.6663434405962856), ((0, 5), 0.26017436694101886), ((0, 6), 0.01212758718041397), ((0, 7), 0.2984343895510508), ((0, 8), 0.09621558179539229), ((0, 9), 0.08893070545042729), ((0, 10), 0.6251961312496821), ((0, 11), 0.41862198729870814), ((0, 12), 0.6496089953556583), ((0, 13), 0.45151329725687167), ((0, 14), 0.2152890917874342), ((0, 15), 0.23383452186059428), ((0, 16), 0.49670850415714823), ((0, 17), 0.2506363615822002), ((0, 18), 0.3899663812042519), ((0, 19), 0.46530502359352055), ((0, 20), 0.5649776003394995), ((0, 21), 0.48529802957065393), ((0, 22), 0.19490426826738308), ((0, 23), 0.5640637646884129), ((0, 24), 0.015789026717942786), ((0, 25), 0.13658816715229713), ((0, 26), 0.5775351598000837), ((0, 27), 0.29540911066889286), ((0, 28), 0.4427360327244187), ((0, 29), 0.46366764703852337), ((0, 30),

True